In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
train = pd.read_csv("/kaggle/input/word2vec-nlp-tutorial/labeledTrainData.tsv.zip", header=0, \
                    delimiter="\t", quoting=3)

print(train.shape)
#print(train.columns.values)
train.columns.values

In [ ]:
#train['review'][0]

In [ ]:
from bs4 import BeautifulSoup
example1=BeautifulSoup(train["review"][0])
print(train["review"][0])
print(example1.get_text)

In [ ]:
import re
# ex1=re.sub("[.]",
#            " ",
#            example1.get_text())

letters_only=re.sub("[^a-zA-Z]",
                   " ",
                   example1.get_text())

print(letters_only)

In [ ]:
lower_case=letters_only.lower()
words=lower_case.split()
print(words)

In [ ]:
import nltk
#nltk.download()



In [ ]:
from nltk.corpus import stopwords
print(stopwords.words("english"))
#print()

In [ ]:
words=[w for w in words if not w in stopwords.words("english")]
print(words)

In [ ]:
def review_to_words(raw_review):
    review_text=BeautifulSoup(raw_review).get_text()
    
    letters_only=re.sub("[^a-zA-Z]"," ",review_text)
    
    words=letters_only.lower().split()
    
    stops=set(stopwords.words("english"))
    
    meaningful_words=[w for w in words if not w in stops]
    
    return (" ".join(meaningful_words))

In [ ]:
clean_review=review_to_words(train["review"][0])
print(clean_review)

In [ ]:
num_reviews=train['review'].size
print ("Cleaning and parsing the training set movie reviews...\n")
clean_train_reviews=[]

for i in range(0,num_reviews):
    if((i+1)%1000==0):
        print("Review %d of %d \n"%(i+1,num_reviews))
    clean_train_reviews.append(review_to_words(train['review'][i]))

In [ ]:
print("Creating the bag of words ... \n")
from sklearn.feature_extraction.text import CountVectorizer

# vectorizer = CountVectorizer(analyzer = "word" , \
#                              tokenizer = None , \
#                              preprocessor = None , \
#                              stop_words = None , \
#                              max_features = 5000)

vectorizer = CountVectorizer(analyzer = "word" , 
                             tokenizer = None , 
                             preprocessor = None , 
                             stop_words = None , 
                             max_features = 5000)


train_data_features = vectorizer.fit_transform(clean_train_reviews)

train_data_features = train_data_features.toarray()

In [ ]:
train_data_features.shape

In [ ]:
vocab = vectorizer.get_feature_names()
print(vocab)

In [ ]:
import collections
count=collections.Counter(vocab)
print(count)

In [ ]:
print("Training the random forest ... ")
from sklearn.ensemble import RandomForestClassifier

forest=RandomForestClassifier(n_estimators = 100)

forest=forest.fit(train_data_features , train["sentiment"])

In [ ]:
test= pd.read_csv("/kaggle/input/word2vec-nlp-tutorial/testData.tsv.zip", header=0,delimiter="\t",
                 quoting=3)

print(test.shape)

num_reviews=len(test["review"])
clean_test_reviews=[]

print("Cleaning and parsing the test set movie reviews ... \n")
for i in range(0,num_reviews) :
    if( (i+1) % 1000 == 0):
        print("Review %d of  %d \n "%(i+1 , num_reviews))
    clean_review = review_to_words(test["review"][i])
    clean_test_reviews.append(clean_review)
    
test_data_features = vectorizer.transform(clean_test_reviews)
test_data_features = test_data_features.toarray()

result=forest.predict(test_data_features)

output=pd.DataFrame(data={"id" : test["id"], "sentiment" : result})

output.to_csv("Bag_of_Words_model.csv", index=False , quoting=3)

